In [1]:
import torch
import torch.nn.functional as F

# Create two tensors that differ by a scale factor
x = torch.tensor([1.0, 2.0, 3.0])
y = x * 2  # y is just x scaled by 2

# Calculate MSE and MAE between x and x
mse = F.mse_loss(x, x)
mae = F.l1_loss(x, x)

# Calculate cosine similarity between x and x
cos_sim = F.cosine_similarity(x.unsqueeze(0), x.unsqueeze(0))

print("Between x and x:")
print(f"MSE: {mse.item():.6f}")
print(f"MAE: {mae.item():.6f}")
print(f"Cosine Similarity: {cos_sim.item():.6f}")

print("\nBetween x and 2x:")
# Calculate MSE and MAE between x and y (2x)
mse = F.mse_loss(x, y)
mae = F.l1_loss(x, y)

# Calculate cosine similarity between x and y
cos_sim = F.cosine_similarity(x.unsqueeze(0), y.unsqueeze(0))

print(f"MSE: {mse.item():.6f}")
print(f"MAE: {mae.item():.6f}")
print(f"Cosine Similarity: {cos_sim.item():.6f}")

# Let's visualize this with normalized vectors
def normalize(tensor):
    return tensor / torch.norm(tensor)

norm_x = normalize(x)
norm_y = normalize(y)

print("\nAfter normalization (x and 2x):")
print(f"x normalized: {norm_x}")
print(f"y normalized: {norm_y}")
print(f"Are they equal after normalization? {torch.allclose(norm_x, norm_y)}")

Between x and x:
MSE: 0.000000
MAE: 0.000000
Cosine Similarity: 1.000000

Between x and 2x:
MSE: 4.666667
MAE: 2.000000
Cosine Similarity: 1.000000

After normalization (x and 2x):
x normalized: tensor([0.2673, 0.5345, 0.8018])
y normalized: tensor([0.2673, 0.5345, 0.8018])
Are they equal after normalization? True


In [1]:
dim = 4096
hidden_dim = 4 * dim
hidden_dim = int(2 * hidden_dim / 3)
ffn_dim_multiplier = 1.3
multiple_of = 1024
if ffn_dim_multiplier is not None:
    hidden_dim = int(ffn_dim_multiplier * hidden_dim)
hidden_dim = multiple_of * ((hidden_dim + multiple_of - 1) // multiple_of)

In [2]:
hidden_dim

14336

In [3]:
import torch
from typing import Union

def compare_matrix_rows(matrix1: torch.Tensor, 
                       matrix2: torch.Tensor) -> bool:
    """
    Compare if two PyTorch tensors contain the same rows regardless of order.
    
    Args:
        matrix1: First matrix as PyTorch tensor
        matrix2: Second matrix as PyTorch tensor
        
    Returns:
        bool: True if matrices contain same rows, False otherwise
        
    Example:
        >>> m1 = torch.tensor([[1, 2], [3, 4]])
        >>> m2 = torch.tensor([[3, 4], [1, 2]])
        >>> compare_matrix_rows(m1, m2)
        True
    """
    # Check if shapes match
    if matrix1.shape != matrix2.shape:
        return False
    
    # Sort both matrices along rows for comparison
    # We use lexicographical sorting
    sorted1, _ = torch.sort(matrix1.view(-1, matrix1.shape[-1]), dim=0)
    sorted2, _ = torch.sort(matrix2.view(-1, matrix2.shape[-1]), dim=0)
    
    # Compare sorted tensors
    return torch.equal(sorted1, sorted2)

# Test cases
def run_tests():
    # Test 1: Same rows, different order
    m1 = torch.tensor([[1, 2], [3, 4]])
    m2 = torch.tensor([[3, 4], [1, 2]])
    assert compare_matrix_rows(m1, m2) == True
    
    # Test 2: Different rows
    m3 = torch.tensor([[1, 2], [3, 4]])
    m4 = torch.tensor([[1, 2], [3, 5]])
    assert compare_matrix_rows(m3, m4) == False
    
    # Test 3: Different shapes
    m5 = torch.tensor([[1, 2], [3, 4]])
    m6 = torch.tensor([[1, 2]])
    assert compare_matrix_rows(m5, m6) == False
    
    # Test 4: Same rows with duplicates
    m7 = torch.tensor([[1, 2], [1, 2]])
    m8 = torch.tensor([[1, 2], [1, 2]])
    assert compare_matrix_rows(m7, m8) == True
    
    # Test 5: Floating point values
    m9 = torch.tensor([[1.1, 2.2], [3.3, 4.4]])
    m10 = torch.tensor([[3.3, 4.4], [1.1, 2.2]])
    assert compare_matrix_rows(m9, m10) == True
    
    print("All tests passed!")

In [4]:
run_tests()

All tests passed!


In [5]:
def compare_matrix_cols(matrix1: torch.Tensor, 
                       matrix2: torch.Tensor) -> bool:
    """
    Compare if two PyTorch tensors contain the same columns regardless of order.
    
    Args:
        matrix1: First matrix as PyTorch tensor
        matrix2: Second matrix as PyTorch tensor
        
    Returns:
        bool: True if matrices contain same columns, False otherwise
        
    Example:
        >>> m1 = torch.tensor([[1, 3], [2, 4]])  # Two columns: [1,2] and [3,4]
        >>> m2 = torch.tensor([[3, 1], [4, 2]])  # Same columns in different order
        >>> compare_matrix_cols(m1, m2)
        True
    """
    # Check if shapes match
    if matrix1.shape != matrix2.shape:
        return False
    
    # Sort both matrices along columns by first transposing
    # This way each column becomes a row that we can sort
    sorted1, _ = torch.sort(matrix1.t(), dim=0)
    sorted2, _ = torch.sort(matrix2.t(), dim=0)
    
    # Compare sorted tensors
    return torch.equal(sorted1, sorted2)

# Test cases
def run_tests():
    # Test 1: Same columns, different order
    m1 = torch.tensor([[1, 3], 
                      [2, 4]])  # Columns are [1,2] and [3,4]
    m2 = torch.tensor([[3, 1], 
                      [4, 2]])  # Columns are [3,4] and [1,2]
    assert compare_matrix_cols(m1, m2) == True
    
    # Test 2: Different columns
    m3 = torch.tensor([[1, 3], 
                      [2, 4]])
    m4 = torch.tensor([[1, 3], 
                      [2, 5]])
    assert compare_matrix_cols(m3, m4) == False
    
    # Test 3: Different shapes
    m5 = torch.tensor([[1, 3], 
                      [2, 4]])
    m6 = torch.tensor([[1], 
                      [2]])
    assert compare_matrix_cols(m5, m6) == False
    
    # Test 4: Same columns with duplicates
    m7 = torch.tensor([[1, 1], 
                      [2, 2]])
    m8 = torch.tensor([[1, 1], 
                      [2, 2]])
    assert compare_matrix_cols(m7, m8) == True
    
    # Test 5: Floating point values
    m9 = torch.tensor([[1.1, 3.3], 
                      [2.2, 4.4]])
    m10 = torch.tensor([[3.3, 1.1], 
                       [4.4, 2.2]])
    assert compare_matrix_cols(m9, m10) == True
    
    # Test 6: Larger matrices
    m11 = torch.tensor([[1, 4, 7],
                       [2, 5, 8],
                       [3, 6, 9]])
    m12 = torch.tensor([[7, 1, 4],
                       [8, 2, 5],
                       [9, 3, 6]])
    assert compare_matrix_cols(m11, m12) == True
    
    print("All tests passed!")
run_tests()

All tests passed!
